In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same',
                    input_shape=(Chans, Samples, 1),
                    use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_regularizer=l2(0.001),
                             padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16),
                             use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    flatten = Flatten()(block2)

    dense = Dense(nb_classes, activation='softmax',
                  kernel_regularizer=l2(0.001))(flatten)

    model = Model(inputs=input1, outputs=dense)
    return model


In [2]:
import pandas as pd

# Load data
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")

# Extract EEG channels
eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X = []
y = []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values
    if data.shape[0] < 1000:  # pad if needed
        data = np.pad(data, ((0, 1000 - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:1000]  # crop
    X.append(data)
    label = group_sorted['label'].iloc[0]
    y.append(label_map[label])

X = np.array(X)  # shape: (n_trials, time_steps, 22)
y = to_categorical(y, num_classes=4)

# Reshape for EEGNet: (n_samples, channels, time, 1)
X = np.transpose(X, (0, 2, 1))  # (samples, channels, time)
X = X[..., np.newaxis]          # (samples, channels, time, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y.argmax(1), random_state=42)


In [3]:
model = EEGNet(nb_classes=4, Chans=22, Samples=1000)
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")


c:\Python\Python365\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step - accuracy: 0.2448 - loss: 1.5828 - val_accuracy: 0.2347 - val_loss: 1.3970
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - accuracy: 0.2614 - loss: 1.4475 - val_accuracy: 0.2500 - val_loss: 1.3962
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.2780 - loss: 1.4272 - val_accuracy: 0.2806 - val_loss: 1.3936
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - accuracy: 0.2712 - loss: 1.4014 - val_accuracy: 0.2832 - val_loss: 1.3895
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.3056 - loss: 1.3966 - val_accuracy: 0.3163 - val_loss: 1.3861
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.3174 - loss: 1.3818 - val_accuracy: 0.3010 - val_loss: 1.3800
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 118ms/step - accuracy: 0.3347 - loss: 1.3667 - val_accuracy: 0.3444 - val_loss: 1.3721
Epoch 8/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.3222 - loss: 1.3703 - val_acc

In [5]:
# Save model to file
model.save("eeg_model.h5")


In [6]:
# Select the first 10 samples from X_test
sample_data = X_test[:10]

# Predict
predictions = model.predict(sample_data)

# Print the raw prediction outputs
print("Raw predictions:\n", predictions)

# Optionally, print the predicted classes
predicted_classes = np.argmax(predictions, axis=1)
print("Predicted classes:", predicted_classes)

# And if you want to compare with true labels:
true_classes = np.argmax(y_test[:10], axis=1)
print("True classes:     ", true_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Raw predictions:
 [[0.4606969  0.13146338 0.2579977  0.149842  ]
 [0.376305   0.23660183 0.16584842 0.2212447 ]
 [0.24468043 0.47660932 0.15276816 0.12594202]
 [0.38998693 0.11366347 0.10394442 0.39240515]
 [0.57750547 0.17519674 0.15199094 0.09530683]
 [0.2361813  0.41088712 0.22748698 0.1254446 ]
 [0.29636675 0.34660792 0.23757693 0.11944833]
 [0.09595322 0.52144843 0.06109606 0.32150227]
 [0.32359543 0.12199177 0.18249795 0.3719149 ]
 [0.19896689 0.46472397 0.2389148  0.09739432]]
Predicted classes: [0 0 1 3 0 1 1 1 3 1]
True classes:      [0 2 1 3 0 1 2 3 3 1]


In [ ]:
'''
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# ---------------------- Filtering Functions ----------------------
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    return butter(order, [low, high], btype='band')

def bandpass_filter_data(data, lowcut=4, highcut=40, fs=250):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return filtfilt(b, a, data, axis=0)

# ---------------------- EEGNet Model ----------------------
def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D, depthwise_regularizer=l2(0.001), padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    flatten = Flatten()(block2)
    dense = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(0.001))(flatten)

    model = Model(inputs=input1, outputs=dense)
    return model

# ---------------------- Load and Preprocess Data ----------------------
# Load CSV
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")

eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X = []
y = []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}
fs = 250  # Sampling frequency
target_length = 1000

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values  # shape: (time_steps, 22)

    # Pad or crop
    if data.shape[0] < target_length:
        data = np.pad(data, ((0, target_length - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:target_length]

    # Z-score normalization per channel
    data = (data - np.mean(data, axis=0)) / (np.std(data, axis=0) + 1e-6)

    X.append(data)
    y.append(label_map[group_sorted['label'].iloc[0]])

# Convert to arrays
X = np.array(X)  # shape: (n_trials, time_steps, channels)
y = to_categorical(y, num_classes=4)

# Reshape for EEGNet: (samples, channels, time, 1)
X = np.transpose(X, (0, 2, 1))  # (samples, channels, time)
X = X[..., np.newaxis]         # (samples, channels, time, 1)

# ---------------------- Train-Test Split ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=np.argmax(y, axis=1), random_state=42
)

# ---------------------- Compute Class Weights ----------------------
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# ---------------------- Train EEGNet ----------------------
model = EEGNet(nb_classes=4, Chans=22, Samples=1000)
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-3), metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=150,
    validation_split=0.2,
    callbacks=[early_stop],
    class_weight=class_weights,
    verbose=1
)

# ---------------------- Evaluate ----------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {acc * 100:.2f}%")'''


In [ ]:
group_sorted['time'].diff().mean()  # Should be ~0.004 → 1/250


0.004

In [ ]:
'''import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# ---------------------- Filtering Functions ----------------------
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    return butter(order, [low, high], btype='band')

def bandpass_filter_data(data, lowcut=8, highcut=30, fs=250):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return filtfilt(b, a, data, axis=0)

# ---------------------- EEGNet Model ----------------------
def EEGNet(
    nb_classes,
    Chans=22,
    Samples=1000,
    dropoutRate1=0.3,
    dropoutRate2=0.4,
    kernLength=64,
    F1=8,
    D=2,
    F2=16,
    norm_rate=0.25
):
    input1 = Input(shape=(Chans, Samples, 1), name='Input')

    # Block 1: Temporal convolution
    block1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False, name='Conv2D_Temporal')(input1)
    block1 = BatchNormalization(name='BN_Temporal')(block1)

    # Block 1: Spatial filtering
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False, depth_multiplier=D,
                             depthwise_regularizer=l2(0.001), padding='valid', name='DepthwiseConv2D_Spatial')(block1)
    block1 = BatchNormalization(name='BN_Spatial')(block1)
    block1 = Activation('relu', name='ReLU_Spatial')(block1)
    block1 = AveragePooling2D((1, 4), name='AvgPool1')(block1)
    block1 = Dropout(dropoutRate1, name='Dropout1')(block1)

    # Block 2: Depthwise Separable Conv
    block2 = SeparableConv2D(F2, (1, 8), use_bias=False, padding='same', name='SeparableConv2D')(block1)
    block2 = BatchNormalization(name='BN_Separable')(block2)
    block2 = Activation('relu', name='ReLU_Separable')(block2)
    block2 = AveragePooling2D((1, 8), name='AvgPool2')(block2)
    block2 = Dropout(dropoutRate2, name='Dropout2')(block2)

    # Classification
    flatten = Flatten(name='Flatten')(block2)
    dense = Dense(nb_classes, activation='softmax',
                  kernel_regularizer=l2(0.001),
                  name='Classifier')(flatten)

    model = Model(inputs=input1, outputs=dense, name='EEGNet_Improved')
    return model

# ---------------------- Load and Preprocess Data ----------------------
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")

eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X = []
y = []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}
fs = 250
target_length = 500  # Reduced for better efficiency

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values

    if data.shape[0] < target_length:
        data = np.pad(data, ((0, target_length - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:target_length]

    # Bandpass filter each trial
    data = bandpass_filter_data(data, 8, 30, fs)

    # Z-score normalization per channel
    data = (data - np.mean(data, axis=0)) / (np.std(data, axis=0) + 1e-6)

    X.append(data)
    y.append(label_map[group_sorted['label'].iloc[0]])

X = np.array(X)  # (samples, time, channels)
y = to_categorical(y, num_classes=4)

# Reshape: (samples, channels, time, 1)
X = np.transpose(X, (0, 2, 1))
X = X[..., np.newaxis]

# ---------------------- Train-Test Split ----------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=np.argmax(y, axis=1), random_state=42
)

# ---------------------- Compute Class Weights ----------------------
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# ---------------------- Train EEGNet ----------------------
model = EEGNet(nb_classes=4, Chans=22, Samples=target_length)

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=150,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    class_weight=class_weights,
    verbose=1
)

# ---------------------- Evaluate ----------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {acc * 100:.2f}%")

'''



Epoch 1/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - accuracy: 0.2490 - loss: 1.5070 - val_accuracy: 0.2730 - val_loss: 1.3953 - learning_rate: 0.0010
Epoch 2/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.2535 - loss: 1.4605 - val_accuracy: 0.2602 - val_loss: 1.3944 - learning_rate: 0.0010
Epoch 3/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.2317 - loss: 1.4579 - val_accuracy: 0.2500 - val_loss: 1.3941 - learning_rate: 0.0010
Epoch 4/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.2659 - loss: 1.4266 - val_accuracy: 0.2219 - val_loss: 1.3952 - learning_rate: 0.0010
Epoch 5/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.2687 - loss: 1.4245 - val_accuracy: 0.2219 - val_loss: 1.3977 - learning_rate: 0.0010
Epoch 6/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.2650 - loss: 1.4050 - val_accuracy: 0.2602 - val_loss: 1.3945 - learning_rate: 0.0010
Epoch 7/150
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.2673 - loss: 1.3999 - 

In [ ]:
'''import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16, norm_rate=0.25):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same',
                    input_shape=(Chans, Samples, 1),
                    use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_regularizer=l2(0.001),
                             padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16),
                             use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropoutRate)(block2)

    flatten = Flatten()(block2)

    dense = Dense(nb_classes, activation='softmax',
                  kernel_regularizer=l2(0.001))(flatten)

    model = Model(inputs=input1, outputs=dense)
    return model

# Parameters
Chans = 22
Samples = 1000
nb_classes = 4
num_samples = 1000  # Total EEG epochs/samples

import pandas as pd

# Load data
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")

# Extract EEG channels
eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X = []
y = []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values
    if data.shape[0] < 1000:  # pad if needed
        data = np.pad(data, ((0, 1000 - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:1000]  # crop
    X.append(data)
    label = group_sorted['label'].iloc[0]
    y.append(label_map[label])

X = np.array(X)  # shape: (n_trials, time_steps, 22)
y = to_categorical(y, num_classes=4)

# Reshape for EEGNet: (n_samples, channels, time, 1)
X = np.transpose(X, (0, 2, 1))  # (samples, channels, time)
X = X[..., np.newaxis]          # (samples, channels, time, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = EEGNet(nb_classes=nb_classes, Chans=Chans, Samples=Samples)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stop], verbose=2)

# Evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Prediction & classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes))
'''

c:\Python\Python365\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
49/49 - 10s - 203ms/step - accuracy: 0.2337 - loss: 1.5563 - val_accuracy: 0.3061 - val_loss: 1.3922
Epoch 2/50
49/49 - 6s - 116ms/step - accuracy: 0.2676 - loss: 1.4548 - val_accuracy: 0.3061 - val_loss: 1.3908
Epoch 3/50
49/49 - 6s - 115ms/step - accuracy: 0.2605 - loss: 1.4424 - val_accuracy: 0.3061 - val_loss: 1.3896
Epoch 4/50
49/49 - 6s - 116ms/step - accuracy: 0.2835 - loss: 1.4194 - val_accuracy: 0.3061 - val_loss: 1.3867
Epoch 5/50
49/49 - 6s - 116ms/step - accuracy: 0.3167 - loss: 1.3847 - val_accuracy: 0.3061 - val_loss: 1.3809
Epoch 6/50
49/49 - 6s - 115ms/step - accuracy: 0.3263 - loss: 1.3711 - val_accuracy: 0.3189 - val_loss: 1.3724
Epoch 7/50
49/49 - 6s - 116ms/step - accuracy: 0.3404 - loss: 1.3569 - val_accuracy: 0.3214 - val_loss: 1.3647
Epoch 8/50
49/49 - 6s - 117ms/step - accuracy: 0.3608 - loss: 1.3411 - val_accuracy: 0.3597 - val_loss: 1.3545
Epoch 9/50
49/49 - 6s - 117ms/step - accuracy: 0.3761 - loss: 1.3218 - val_accuracy: 0.3469 - val_loss: 1.3366


In [ ]:
print(f"Test Accuracy: {acc * 100:.2f}%")

Test Accuracy: 51.02%
